In [1]:
import tensorflow as tf

import pandas as pd
import codecs
import numpy as np
import math
import csv
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import recall_score, precision_score # New
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
import tensorflow.keras.preprocessing
#from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Add, Multiply, Subtract
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, BatchNormalization
# regularizers
from tensorflow.keras.layers import Conv1D, Conv2D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dropout, SpatialDropout1D
from scipy import signal
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pylab as plt
import matplotlib.dates as mdates
import seaborn as sns

from keijzer import *
from tensorflow import keras
from keras.models import Model
from keras.layers import *
from tensorflow.keras.optimizers import Adam,SGD
from keras.layers import Dense, Input, Flatten, Add, concatenate, Dropout, Activation, Multiply, Embedding, Layer, Reshape
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D  
from keras.layers import Conv1D, AveragePooling1D, MaxPooling1D
#from keras.ops import convert_to_tensor, convert_to_numpy
#from keras.utils import plot_model
from keras import activations
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
import math

def get_flops(model):
    run_meta = tf.RunMetadata()
    opts = tf.profiler.ProfileOptionBuilder.float_operation()

    # We use the Keras session graph in the call to the profiler.
    flops = tf.profiler.profile(graph=K.get_session().graph,
                                run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops  # Prints the "flops" of the model.

class LossHistory(tensorflow.keras.callbacks.Callback):  # history = LossHistory()
    def init(self):  # history.init()
        self.losses = []
        # self.accs = []
        self.val_losses = []
        # self.val_accs = []
        self.rmses = []
        self.mses = []
        self.maes = []
        self.mapes = []
        self.val_rmses = []
        self.val_mses = []
        self.val_maes = []
        self.val_mapes = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        # self.accs.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        # self.val_accs.append(logs.get('val_accuracy
        self.rmses.append(logs.get('root_mean_squared_error'))
        self.mses.append(logs.get('mean_squared_error'))
        self.maes.append(logs.get('mean_absolute_error'))
        self.mapes.append(logs.get('mean_absolute_percentage_error'))
        self.val_rmses.append(logs.get('val_root_mean_squared_error'))
        self.val_mses.append(logs.get('val_mean_squared_error'))
        self.val_maes.append(logs.get('val_mean_absolute_error'))
        self.val_mapes.append(logs.get('val_mean_absolute_percentage_error'))

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0' # 여러개 사용시 '0,1,2' 식으로 하나의 문자열에 입력
gpus = tf.config.experimental.list_physical_devices('GPU') # 호스트 러나임에 표시되는 GPU 장치 목록 반환

if gpus: # 반환된 GPU 장치 목록이 있다면
    try: # 해당 장치에 대한 메모리 증가 활성화 여부 설정
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e: # try문 실패시에 에러문구 출력
        print(e)

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2020.csv'
data_source1 = pd.read_csv(data_path)
dat_source1 = pd.DataFrame(data_source1)
#dat_source1 = dat_source1.iloc[:-3,:]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2021.csv'
data_source2 = pd.read_csv(data_path)
dat_source2 = pd.DataFrame(data_source2)
dat_source2 = dat_source2.iloc[:8761,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2022.csv'
data_source3 = pd.read_csv(data_path)
dat_source3 = pd.DataFrame(data_source3)
dat_source3 = dat_source3.iloc[:8760,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2023.csv'
data_source4 = pd.read_csv(data_path)
dat_source4 = pd.DataFrame(data_source4)
dat_source4 = dat_source4.iloc[:8760,:9]

dat_arr1 = np.array(dat_source1)
dat_arr2 = np.array(dat_source2)
dat_arr3 = np.array(dat_source3)
dat_arr4 = np.array(dat_source4)

dat_arr = np.concatenate([dat_arr1, dat_arr2, dat_arr3, dat_arr4],axis=0)
dat_arr.shape # 8784, 8761, 8760, 8760

dat_source = pd.DataFrame(dat_arr, columns=['Time-Date stamp','Date','ERCOT Load','Total Wind Output','Total Wind Installed','Wind Output, % of Load','Wind Output, % of Installed','1-hr MW change','1-hr % change'])
dat_source

,Time-Date stamp,Date,ERCOT Load,Total Wind Output,Total Wind Installed,"Wind Output, % of Load","Wind Output, % of Installed",1-hr MW change,1-hr % change
0,01-01-2020 00:00:00,Jan-01,36823,12335,27040,33.5,45.617,NaN,NaN
1,01-01-2020 01:00:00,Jan-01,36511,13300,27040,36.4,49.187,965.4,7.826
2,01-01-2020 02:00:00,Jan-01,36022,13586,27040,37.7,50.245,286.2,2.152
3,01-01-2020 03:00:00,Jan-01,35790,14107,27040,39.4,52.171,520.9,3.834
4,01-01-2020 04:00:00,Jan-01,35642,14887,27040,41.8,55.055,779.6,5.527
...,...,...,...,...,...,...,...,...,...
35060,12-31-2023 20:00:00,Dec-31,44887,17161,38631,38.23,44.42,2858,19.99
35061,12-31-2023 21:00:00,Dec-31,44013,18301,38631,41.58,47.37,1139,6.64
35062,12-31-2023 22:00:00,Dec-31,43280,18607,38631,42.99,48.17,307,1.67
35063,12-31-2023 23:00:00,Dec-31,42319,17866,38631,42.22,46.25,-741,-3.98


In [4]:
dat = pd.DataFrame()
dat['MW % change'] = np.array(dat_source.iloc[1:,3], dtype=float)-np.array(dat_source.iloc[0:-1,3], dtype=float)
dat['MW % change'] = np.divide(np.array(dat['MW % change']),np.array(dat_source.iloc[1:,4]))*100
dat['Total Wind Output'] = np.divide(np.array(dat_source.iloc[1:,3]), np.array(dat_source.iloc[1:,4]))

dat

,MW % change,Total Wind Output
0,3.56879,0.491864
1,1.05769,0.502441
2,1.92678,0.521709
3,2.88462,0.550555
4,-0.580621,0.544749
...,...,...
35059,7.3982,0.444229
35060,2.951,0.473739
35061,0.79211,0.48166
35062,-1.91815,0.462478


## Figures

In [5]:
ramp_ratio = 20
load_ratio = 10
year_arr = np.concatenate([np.ones(8784)*2020, np.ones(8760)*2021, np.ones(8760)*2022, np.ones(8760)*2023])
year_df = pd.DataFrame(year_arr, columns=['Year'], dtype=int)
wind_df = pd.concat([year_df, dat], axis=1)
wind_df

,Year,MW % change,Total Wind Output
0,2020,3.56879,0.491864
1,2020,1.05769,0.502441
2,2020,1.92678,0.521709
3,2020,2.88462,0.550555
4,2020,-0.580621,0.544749
...,...,...,...
35059,2023,7.3982,0.444229
35060,2023,2.951,0.473739
35061,2023,0.79211,0.48166
35062,2023,-1.91815,0.462478


In [6]:
#from tensorflow.keras.layers import preprocessing
#from tensorflow.keras import preprocessing
import sklearn

wind_df = wind_df.reset_index(drop=True)
#std_scaler = sklearn.preprocessing.StandardScaler()
#std_scaler_load = sklearn.preprocessing.StandardScaler()
#std_scaler_ramp = sklearn.preprocessing.StandardScaler()

std_scaler = sklearn.preprocessing.MinMaxScaler()
std_scaler_load = sklearn.preprocessing.MinMaxScaler()
std_scaler_ramp = sklearn.preprocessing.MinMaxScaler()

wind_df[['Total Wind Output']] = std_scaler.fit_transform(wind_df[['Total Wind Output']])
wind_df[['MW % change']] = std_scaler_ramp.fit_transform(wind_df[['MW % change']])
#wind_df[['Wind Change, % of Load', 'Wind Change']] = std_scaler_ramp.fit_transform(wind_df[['Wind Change, % of Load', 'Wind Change']])
wind_df = wind_df.iloc[:-1,:]
wind_df

,Year,MW % change,Total Wind Output
0,2020,0.620197,0.644724
1,2020,0.544580,0.658617
2,2020,0.570751,0.683924
3,2020,0.599594,0.721813
4,2020,0.495246,0.714187
...,...,...,...
35058,2023,0.774643,0.484984
35059,2023,0.735512,0.582157
35060,2023,0.601593,0.620917
35061,2023,0.536583,0.631322


In [7]:
ramp_df = pd.DataFrame(wind_df)
ramp_df.columns=['Year', 'Wind Change', 'Wind Output']
ramp_df

,Year,Wind Change,Wind Output
0,2020,0.620197,0.644724
1,2020,0.544580,0.658617
2,2020,0.570751,0.683924
3,2020,0.599594,0.721813
4,2020,0.495246,0.714187
...,...,...,...
35058,2023,0.774643,0.484984
35059,2023,0.735512,0.582157
35060,2023,0.601593,0.620917
35061,2023,0.536583,0.631322


## Common Definitions

In [8]:
def create_dataset(dataset, timesteps, output_timesteps, leadtime):
    dataX, dataY = [], []
    for i in range(len(dataset) - timesteps -output_timesteps - leadtime - 1):
        a = dataset[i:(i + timesteps), :]
        dataX.append(a)
        dataY.append(dataset[(i + timesteps + leadtime):(i+timesteps+output_timesteps+leadtime), :])
    return np.array(dataX), np.array(dataY)

def svdadd(X, start, end, b_size):
    dec_x = X[start:end, :, :]
    U, S, Vh = svd(dec_x.reshape(b_size, -1), full_matrices=True)

    high_sig = np.matmul(np.matmul(U[:,:high_ind], np.diag(S[0:high_ind])), Vh[:high_ind,:])
    low_sig = X[:dec_num, :, :].reshape(b_size,-1)-high_sig

    rec_x = np.zeros((b_size, b_size, high_ind))
    
    #print(sum(S[0:high_ind])/sum(S))
    
    for i in range(high_ind):
        rec_x[:,:,i] = np.matmul((U[:,i]*S[i]).reshape(-1,1), Vh[i,:].reshape(1,-1))    
    return rec_x, low_sig

In [9]:
from sklearn.model_selection import train_test_split
from numpy.linalg import svd
import gc

def MAEMS(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true))*100

def MAEMD(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true-K.mean(y_true)))*100

def mse_mae(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))+K.mean(K.abs(y_pred - y_true))

from keras.layers.advanced_activations import PReLU

## Wind Generation Forecasting

In [10]:
norm_df = np.array(wind_df.iloc[:,2]).reshape(-1,1)
timesteps = 24*6
output_timesteps = 12
leadtime = 4
num_features = 1
X, Y = create_dataset(norm_df, timesteps, output_timesteps, leadtime)
b_size = timesteps

In [11]:
dec_num = timesteps
high_ind = 35

addX = np.zeros((X.shape[0], dec_num, high_ind+1))

for k in range(X.shape[0]//dec_num):
    rec_x, low_sig = svdadd(X, k*dec_num, (k+1)*dec_num, b_size)
    addX[k*dec_num:k*dec_num+dec_num,:, :-1] = rec_x
    addX[k*dec_num:k*dec_num+dec_num,:, -1] = low_sig
    
rec_x, low_sig = svdadd(X, X.shape[0]-dec_num, X.shape[0], b_size)
addX[X.shape[0]-dec_num:X.shape[0], :, :-1] = rec_x
addX[X.shape[0]-dec_num:X.shape[0],:, -1] = low_sig

addX_det = addX[:, :, :-1]
addX_det = np.sum(addX_det, axis=2)
X = np.concatenate([X, addX], axis=2)
num_features = X.shape[2]

## Load and Test Pre-trained Basic Model

In [12]:
trvaX, teX, trvaY, teY = train_test_split(X, Y, test_size=0.1, shuffle=False)
trvaY = trvaY.reshape(-1,output_timesteps)
teY = teY.reshape(-1,output_timesteps)
trvaX.shape, trvaY.shape, teX.shape, teY.shape

((31411, 144, 37), (31411, 12), (3491, 144, 37), (3491, 12))

In [13]:
with tf.device('/gpu:0'):
    wind_model = keras.models.load_model('Basic Model Final_fh12.h5', custom_objects={'MAEMS': MAEMS})
    winder_model = keras.models.load_model('Error Learning Model_fh12.h5', custom_objects={'mse_mae': mse_mae})
    gc.collect()
    trainPredict = wind_model.predict(trvaX, batch_size=b_size)
    testPredict = wind_model.predict(teX, batch_size=b_size)

In [14]:
def npMAEMS(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true))*100
def npMAEMD(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true-np.mean(y_true)))*100
def npMSE(y_true, y_pred):
    return np.mean(np.square(-y_true+y_pred))
def npMAE(y_true, y_pred):
    return np.mean(abs(-y_true+y_pred))

In [15]:
travaY = trvaY.reshape(-1)
trPredict = trainPredict.reshape(-1)
testY = teY.reshape(-1)
tePredict = testPredict.reshape(-1)

In [16]:
npMSE(trPredict, travaY), npMAE(trPredict, travaY), npMAEMS(trPredict, travaY), npMSE(tePredict, testY), npMAE(tePredict, testY), npMAEMS(tePredict, testY)

(0.0046658855261013776,
 0.04610948451513378,
 0.9110652813778538,
 0.027214180424758545,
 0.12977480004830266,
 3.0918946475182545)

In [17]:
trPredict = trPredict[(travaY>0.8)]
travaY = travaY[(travaY>0.8)]
tePredict = tePredict[(testY>0.8)]
testY = testY[(testY>0.8)]
npMSE(trPredict, travaY), npMAE(trPredict, travaY), npMAEMS(trPredict, travaY), npMSE(tePredict, testY), npMAE(tePredict, testY), npMAEMS(tePredict, testY)

(0.0005979944479619424,
 0.018733007316249363,
 1.3459041753951,
 0.02415418582053564,
 0.11846375994115907,
 4.7224328054458615)

In [18]:
errors = trainPredict - trvaY
prediction = trainPredict

In [19]:
norm_df2 = pd.DataFrame(norm_df).iloc[timesteps:trvaX.shape[0]+timesteps, :]
norm_df2.columns = ['Normalized Wind']
norm_df2 = norm_df2.reset_index(drop=True)
norm_df2.shape

(31411, 1)

## Load and Test Pre-trained FFEL Model

In [20]:
Prindex = ['Prediction1', 'Prediction2','Prediction3','Prediction4','Prediction5','Prediction6','Prediction7','Prediction8','Prediction9','Prediction10','Prediction11','Prediction12']
Erindex = ['Error1', 'Error2','Error3','Error4','Error5','Error6','Error7','Error8','Error9','Error10','Error11','Error12']

pr_df = pd.DataFrame(prediction, columns=Prindex)
er_df = pd.DataFrame(errors, columns=Erindex)
norm_df2 = pd.concat([norm_df2, pr_df, er_df],axis=1)
norm_df2

,Normalized Wind,Prediction1,Prediction2,Prediction3,Prediction4,Prediction5,Prediction6,Prediction7,Prediction8,Prediction9,...,Error3,Error4,Error5,Error6,Error7,Error8,Error9,Error10,Error11,Error12
0,0.164171,0.399960,0.429434,0.481648,0.506049,0.511644,0.536437,0.559587,0.591610,0.606001,...,0.116619,0.105075,0.110718,0.180638,0.171145,0.115393,0.091944,0.096135,0.104033,0.092794
1,0.195842,0.352472,0.371041,0.411366,0.421782,0.430169,0.470344,0.501786,0.537461,0.559749,...,0.010391,0.020856,0.074369,0.081902,0.025568,0.023403,0.045740,0.060027,0.062624,0.084380
2,0.228047,0.407803,0.409733,0.429213,0.432311,0.450604,0.495748,0.518091,0.536294,0.527751,...,0.028287,0.076511,0.062162,0.019530,0.004033,0.022285,0.030258,0.031529,0.058181,0.057732
3,0.260835,0.441920,0.433120,0.450634,0.455771,0.483881,0.536042,0.559484,0.561809,0.509127,...,0.094834,0.067329,0.007663,0.021984,0.045475,0.064316,0.031258,0.054922,0.075313,-0.034730
4,0.277934,0.447935,0.463115,0.499792,0.510718,0.532205,0.570618,0.559038,0.531497,0.481462,...,0.111349,0.034501,0.018148,0.056609,0.061545,0.053628,0.055909,0.082757,-0.020529,-0.042525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31406,0.253414,0.518468,0.569279,0.619652,0.631811,0.605485,0.606846,0.582167,0.572136,0.550121,...,0.031003,0.003244,-0.008893,0.067457,0.138510,0.162260,0.055181,0.004557,0.011152,-0.010587
31407,0.272763,0.577464,0.630980,0.632098,0.594880,0.545163,0.544239,0.518164,0.524765,0.517756,...,0.003531,-0.019498,0.005774,0.100581,0.108288,0.029825,-0.009362,0.017617,0.009080,-0.039252
31408,0.323174,0.612628,0.634992,0.605043,0.561689,0.532853,0.536198,0.509705,0.517685,0.499898,...,-0.009335,0.022300,0.089195,0.126322,0.014766,-0.009432,0.002379,-0.007620,-0.026601,0.004391
31409,0.382300,0.653783,0.649182,0.596476,0.539113,0.517933,0.539815,0.538498,0.539775,0.502985,...,0.057086,0.095456,0.108057,0.044875,0.011381,0.042255,0.010415,-0.002427,0.055842,0.081402


In [21]:
norm_df2 = np.array(norm_df2)
Xe, Ye = create_dataset(norm_df2, timesteps, 1, 0)
Xe.shape

(31265, 144, 25)

In [22]:
trvaXe = Xe
trvaYe = Ye[:,:,-output_timesteps:].reshape(-1,output_timesteps)

In [23]:
with tf.device('/gpu:0'):
    trainerPredict = winder_model.predict(trvaXe, batch_size=b_size)

In [24]:
addtrainPredict = trainPredict[timesteps:-2,:]-trainerPredict
#addtestPredict = testPredict[timesteps:-2,:]-testerPredict

In [25]:
errors = testPredict - teY
prediction = testPredict

norm_df2 = pd.DataFrame(norm_df).iloc[trvaX.shape[0]+timesteps:trvaX.shape[0]+timesteps+teY.shape[0], :]
norm_df2.columns = ['Normalized Wind']
norm_df2 = norm_df2.reset_index(drop=True)

Prindex = ['Prediction1', 'Prediction2','Prediction3','Prediction4','Prediction5','Prediction6','Prediction7','Prediction8','Prediction9','Prediction10','Prediction11','Prediction12']
Erindex = ['Error1', 'Error2','Error3','Error4','Error5','Error6','Error7','Error8','Error9','Error10','Error11','Error12']

pr_df = pd.DataFrame(prediction, columns=Prindex)
er_df = pd.DataFrame(errors, columns=Erindex)

norm_df2 = pd.concat([norm_df2, pr_df, er_df],axis=1)
norm_df2 = np.array(norm_df2)
output_timesteps = 1
Xe, Ye = create_dataset(norm_df2, timesteps, output_timesteps,0)
teXe = Xe
teYe = Ye[:,:,-output_timesteps:].reshape(-1,output_timesteps)

In [26]:
with tf.device('/gpu:0'):
    #trainerPredict = winder_model.predict(trvaXe, batch_size=b_size)
    testerPredict = winder_model.predict(teXe, batch_size=b_size)

In [27]:
#addtrainPredict = trainPredict[timesteps:-2,:]-trainerPredict
addtestPredict = testPredict[timesteps:-2,:]-testerPredict

In [28]:
print('Error Train Score > MSE == ', (np.mean(np.square(trvaY[timesteps:-2,:]-trainPredict[timesteps:-2,:]))), ' MAE == ', mean_absolute_error(trvaY[timesteps:-2,:], trainPredict[timesteps:-2,:]), ' MAPE == ', mean_absolute_percentage_error(trvaY[timesteps:-2,:], trainPredict[timesteps:-2,:]), )
print('Error Train Score > MSE == ', (np.mean(np.square(trvaY[timesteps:-2,:]-addtrainPredict))), ' MAE == ', mean_absolute_error(trvaY[timesteps:-2,:], addtrainPredict), ' MAPE == ', mean_absolute_percentage_error(trvaY[timesteps:-2,:], addtrainPredict))
print('Error Test Score > MSE == ', (np.mean(np.square(teY[timesteps:-2,:]-testPredict[timesteps:-2,:]))), ' MAE == ', mean_absolute_error(teY[timesteps:-2,:], testPredict[timesteps:-2,:]), ' MAPE == ', mean_absolute_percentage_error(teY[timesteps:-2,:], testPredict[timesteps:-2,:]), ' MAEMS == ', np.mean((abs(testPredict[timesteps:-2,:] - teY[timesteps:-2,:]))*np.square(teY[timesteps:-2,:]-np.mean(teY[timesteps:-2,:])))*100)
print('Error Test Score > MSE == ', (np.mean(np.square(teY[timesteps:-2,:]-addtestPredict))), ' MAE == ', mean_absolute_error(teY[timesteps:-2,:], addtestPredict), ' MAPE == ', mean_absolute_percentage_error(teY[timesteps:-2,:], addtestPredict), ' MAEMS == ', np.mean((abs(addtestPredict - teY[timesteps:-2,:]))*np.square(teY[timesteps:-2,:]-np.mean(teY[timesteps:-2,:])))*100)

Error Train Score > MSE ==  0.004678756598170035  MAE ==  0.04617222544271949  MAPE ==  inf
Error Train Score > MSE ==  0.0007931533095822846  MAE ==  0.020702294431063552  MAPE ==  inf
Error Test Score > MSE ==  0.027476862423996988  MAE ==  0.13036319819526196  MAPE ==  76.09992741891799  MAEMS ==  0.5963107028853325
Error Test Score > MSE ==  0.017314232230475065  MAE ==  0.09909626964061886  MAPE ==  39.937376048426316  MAEMS ==  0.46114252727216004


In [29]:
result_tr = np.concatenate([addtrainPredict.reshape(-1,1), trvaY[timesteps:-2,:].reshape(-1,1), trvaY[timesteps:-2,:].reshape(-1,1)-addtrainPredict.reshape(-1,1), trainPredict[timesteps:-2,:].reshape(-1,1)], axis=1)
result_te = np.concatenate([addtestPredict.reshape(-1,1), teY[timesteps:-2,:].reshape(-1,1), teY[timesteps:-2,:].reshape(-1,1)-addtestPredict.reshape(-1,1), testPredict[timesteps:-2,:].reshape(-1,1)], axis=1)

result_tr_df = pd.DataFrame(result_tr, columns=['Predicted Value','Real Value','Forecasting Error', 'Initial Predicted Value'])
result_te_df = pd.DataFrame(result_te, columns=['Predicted Value','Real Value','Forecasting Error', 'Initial Predicted Value'])

In [30]:
teY[timesteps:-2,:].reshape(-1,1).shape, addtestPredict.reshape(-1,1).shape

((40140, 1), (40140, 1))

In [31]:
result_te_df

,Predicted Value,Real Value,Forecasting Error,Initial Predicted Value
0,0.628161,0.715967,0.087806,0.678872
1,0.553593,0.686438,0.132845,0.648870
2,0.507566,0.641152,0.133587,0.624324
3,0.490559,0.553299,0.062741,0.611558
4,0.428668,0.487758,0.059090,0.617402
...,...,...,...,...
40135,0.173075,0.293595,0.120521,0.456053
40136,0.220378,0.311174,0.090796,0.440937
40137,0.222472,0.341026,0.118554,0.402120
40138,0.181430,0.370742,0.189313,0.396793


In [32]:
error_tr_FFEL = trvaY[timesteps:-2,:].reshape(-1,1)-addtrainPredict.reshape(-1,1)
error_tr_basic = trvaY[timesteps:-2,:].reshape(-1,1)-trainPredict[timesteps:-2,:].reshape(-1,1)
error_te_FFEL = teY[timesteps:-2,:].reshape(-1,1)-addtestPredict.reshape(-1,1)
error_te_basic = teY[timesteps:-2,:].reshape(-1,1)-testPredict[timesteps:-2,:].reshape(-1,1)

In [33]:
np.mean(error_tr_FFEL), np.var(error_tr_FFEL), np.mean(error_tr_basic), np.var(error_tr_basic), np.mean(error_te_FFEL),np.var(error_te_FFEL), np.mean(error_te_basic), np.var(error_te_basic)

(0.0009620649529529979,
 0.0007922277406085841,
 -0.03537846783132707,
 0.0034271206120777913,
 0.07489673148494132,
 0.011704711843347666,
 -0.07365980066153373,
 0.022051096190500104)

In [34]:
result_tr_df.to_csv('training_result_df_fh12.csv')
result_te_df.to_csv('test_result_df_fh12.csv')

# Ramp Rate Forecasting

In [35]:
norm_df = np.array(wind_df.iloc[:,1]).reshape(-1,1)
timesteps = 24*6
output_timesteps = 12
leadtime = 4
num_features = 1
X, Y = create_dataset(norm_df, timesteps, output_timesteps, leadtime)
b_size = timesteps

In [36]:
X.shape, Y.shape

((34902, 144, 1), (34902, 12, 1))

In [37]:
dec_num = timesteps
high_ind = 35

addX = np.zeros((X.shape[0], dec_num, high_ind+1))

for k in range(X.shape[0]//dec_num):
    rec_x, low_sig = svdadd(X, k*dec_num, (k+1)*dec_num, b_size)
    addX[k*dec_num:k*dec_num+dec_num,:, :-1] = rec_x
    addX[k*dec_num:k*dec_num+dec_num,:, -1] = low_sig
    
rec_x, low_sig = svdadd(X, X.shape[0]-dec_num, X.shape[0], b_size)
addX[X.shape[0]-dec_num:X.shape[0], :, :-1] = rec_x
addX[X.shape[0]-dec_num:X.shape[0],:, -1] = low_sig

addX_det = addX[:, :, :-1]
addX_det = np.sum(addX_det, axis=2)
X = np.concatenate([X, addX], axis=2)
num_features = X.shape[2]

## Load and Test Pre-trained Basic Model

In [38]:
trvaX, teX, trvaY, teY = train_test_split(X, Y, test_size=0.1, shuffle=False)
trvaY = trvaY.reshape(-1,output_timesteps)
teY = teY.reshape(-1,output_timesteps)
trvaX.shape, trvaY.shape, teX.shape, teY.shape

((31411, 144, 37), (31411, 12), (3491, 144, 37), (3491, 12))

In [39]:
gc.collect()

4185

In [40]:
with tf.device('/gpu:0'):
    ramp_model = keras.models.load_model('Basic Ramp Model Final_fh12.h5', custom_objects={'MAEMD': MAEMD})
    ramper_model = keras.models.load_model('Error Learning Ramp Model_fh12.h5', custom_objects={'mse_mae': mse_mae})
    gc.collect()
    trainPredict = ramp_model.predict(trvaX, batch_size=b_size)
    testPredict = ramp_model.predict(teX, batch_size=b_size)

In [41]:
def npMAEMS(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true))*100
def npMAEMD(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true-np.mean(y_true)))*100
def npMSE(y_true, y_pred):
    return np.mean(np.square(-y_true+y_pred))
def npMAE(y_true, y_pred):
    return np.mean(abs(-y_true+y_pred))

In [42]:
errors = trainPredict - trvaY
prediction = trainPredict

In [43]:
norm_df2 = pd.DataFrame(norm_df).iloc[timesteps:trvaX.shape[0]+timesteps, :]
norm_df2.columns = ['Normalized Wind']
norm_df2 = norm_df2.reset_index(drop=True)
norm_df2.shape

(31411, 1)

## Load and Test Pre-trained FFEL Model

In [44]:
Prindex = ['Prediction1', 'Prediction2','Prediction3','Prediction4','Prediction5','Prediction6','Prediction7','Prediction8','Prediction9','Prediction10','Prediction11','Prediction12']
Erindex = ['Error1', 'Error2','Error3','Error4','Error5','Error6','Error7','Error8','Error9','Error10','Error11','Error12']

pr_df = pd.DataFrame(prediction, columns=Prindex)
er_df = pd.DataFrame(errors, columns=Erindex)
norm_df2 = pd.concat([norm_df2, pr_df, er_df],axis=1)
norm_df2

,Normalized Wind,Prediction1,Prediction2,Prediction3,Prediction4,Prediction5,Prediction6,Prediction7,Prediction8,Prediction9,...,Error3,Error4,Error5,Error6,Error7,Error8,Error9,Error10,Error11,Error12
0,0.593915,0.576944,0.552743,0.563341,0.493121,0.408325,0.406069,0.521226,0.553002,0.515943,...,-0.049394,-0.102019,-0.104293,-0.003204,-0.066340,-0.160963,-0.083540,0.055031,0.053184,-0.043365
1,0.585340,0.538577,0.550591,0.492083,0.416955,0.389544,0.506517,0.521203,0.517316,0.591400,...,-0.103057,-0.095664,-0.019728,-0.081050,-0.192763,-0.082167,0.078781,0.171590,0.051935,0.026303
2,0.586565,0.549176,0.526222,0.433467,0.398326,0.593109,0.592674,0.520525,0.508530,0.499884,...,-0.079152,-0.010947,0.005542,-0.121292,-0.078958,-0.004089,0.025018,-0.032248,-0.006049,-0.112257
3,0.587901,0.538702,0.467881,0.471990,0.658254,0.617754,0.497131,0.489012,0.455205,0.452697,...,0.062717,0.070687,-0.096211,-0.102352,-0.023607,-0.019661,-0.015041,0.028827,-0.006023,-0.213279
4,0.551930,0.384145,0.436745,0.623044,0.571493,0.391706,0.415081,0.515201,0.457060,0.404837,...,0.035477,-0.142473,-0.207777,-0.097538,0.040335,-0.010679,0.012047,-0.009221,-0.194492,0.035325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31406,0.527916,0.622352,0.684375,0.695968,0.778954,0.746770,0.570206,0.381604,0.400599,0.312626,...,0.013395,0.174708,0.266570,0.229397,0.088351,-0.034683,-0.395123,-0.243862,-0.044068,-0.120413
31407,0.557089,0.581286,0.647596,0.628323,0.565862,0.510710,0.443483,0.490750,0.420683,0.351688,...,0.024077,0.085661,0.169901,0.150230,0.055469,-0.287067,-0.234813,-0.030287,-0.058603,-0.143870
31408,0.628303,0.595082,0.636028,0.584006,0.463814,0.386808,0.378333,0.440461,0.414857,0.438971,...,0.103806,0.123005,0.093555,-0.056949,-0.267289,-0.171645,-0.005902,-0.030069,-0.171162,-0.154822
31409,0.648285,0.733841,0.687098,0.559241,0.401101,0.386054,0.375634,0.363663,0.421679,0.443951,...,0.218432,0.107848,-0.049227,-0.332115,-0.222838,-0.023194,-0.057429,-0.104896,0.030386,0.088422


In [45]:
norm_df2 = np.array(norm_df2)
output_timesteps = 1
Xe, Ye = create_dataset(norm_df2, timesteps, output_timesteps,0)
Xe.shape

(31265, 144, 25)

In [46]:
trvaXe = Xe
trvaYe = Ye[:,:,-output_timesteps:].reshape(-1,output_timesteps)

In [47]:
with tf.device('/gpu:0'):
    trainerPredict = ramper_model.predict(trvaXe, batch_size=b_size)

In [48]:
addtrainPredict = trainPredict[timesteps:-2,:]-trainerPredict
#addtestPredict = testPredict[timesteps:-2,:]-testerPredict

In [49]:
errors = testPredict - teY
prediction = testPredict

norm_df2 = pd.DataFrame(norm_df).iloc[trvaX.shape[0]+timesteps:trvaX.shape[0]+timesteps+teY.shape[0], :]
norm_df2.columns = ['Normalized Wind']
norm_df2 = norm_df2.reset_index(drop=True)

Prindex = ['Prediction1', 'Prediction2','Prediction3','Prediction4','Prediction5','Prediction6','Prediction7','Prediction8','Prediction9','Prediction10','Prediction11','Prediction12']
Erindex = ['Error1', 'Error2','Error3','Error4','Error5','Error6','Error7','Error8','Error9','Error10','Error11','Error12']

pr_df = pd.DataFrame(prediction, columns=Prindex)
er_df = pd.DataFrame(errors, columns=Erindex)

norm_df2 = pd.concat([norm_df2, pr_df, er_df],axis=1)
norm_df2 = np.array(norm_df2)
output_timesteps = 1
Xe, Ye = create_dataset(norm_df2, timesteps, output_timesteps,0)
teXe = Xe
teYe = Ye[:,:,-output_timesteps:].reshape(-1,output_timesteps)

In [50]:
with tf.device('/gpu:0'):
    #trainerPredict = winder_model.predict(trvaXe, batch_size=b_size)
    testerPredict = ramper_model.predict(teXe, batch_size=b_size)

In [51]:
#addtrainPredict = trainPredict[timesteps:-2,:]-trainerPredict
addtestPredict = testPredict[timesteps:-2,:]-testerPredict

In [52]:
print('Error Train Score > MSE == ', (np.mean(np.square(trvaY[timesteps:-2,:]-trainPredict[timesteps:-2,:]))), ' MAE == ', mean_absolute_error(trvaY[timesteps:-2,:], trainPredict[timesteps:-2,:]), ' MAPE == ', mean_absolute_percentage_error(trvaY[timesteps:-2,:], trainPredict[timesteps:-2,:]), )
print('Error Train Score > MSE == ', (np.mean(np.square(trvaY[timesteps:-2,:]-addtrainPredict))), ' MAE == ', mean_absolute_error(trvaY[timesteps:-2,:], addtrainPredict), ' MAPE == ', mean_absolute_percentage_error(trvaY[timesteps:-2,:], addtrainPredict))
print('Error Test Score > MSE == ', (np.mean(np.square(teY[timesteps:-2,:]-testPredict[timesteps:-2,:]))), ' MAE == ', mean_absolute_error(teY[timesteps:-2,:], testPredict[timesteps:-2,:]), ' MAPE == ', mean_absolute_percentage_error(teY[timesteps:-2,:], testPredict[timesteps:-2,:]), ' MAEMS == ', np.mean((abs(testPredict[timesteps:-2,:] - teY[timesteps:-2,:]))*np.square(teY[timesteps:-2,:]-np.mean(teY[timesteps:-2,:])))*100)
print('Error Test Score > MSE == ', (np.mean(np.square(teY[timesteps:-2,:]-addtestPredict))), ' MAE == ', mean_absolute_error(teY[timesteps:-2,:], addtestPredict), ' MAPE == ', mean_absolute_percentage_error(teY[timesteps:-2,:], addtestPredict), ' MAEMS == ', np.mean((abs(addtestPredict - teY[timesteps:-2,:]))*np.square(teY[timesteps:-2,:]-np.mean(teY[timesteps:-2,:])))*100)

Error Train Score > MSE ==  0.01204673097906137  MAE ==  0.08609349542983774  MAPE ==  inf
Error Train Score > MSE ==  0.0037042543856255316  MAE ==  0.047405304788622316  MAPE ==  inf
Error Test Score > MSE ==  0.012836964020431156  MAE ==  0.08934994886853158  MAPE ==  18.053389353537742  MAEMS ==  0.09444103488602
Error Test Score > MSE ==  0.004368571366525524  MAE ==  0.05136803933848779  MAPE ==  10.427107213341527  MAEMS ==  0.05046912748215309


In [53]:
result_tr = np.concatenate([addtrainPredict.reshape(-1,1), trvaY[timesteps:-2,:].reshape(-1,1), trvaY[timesteps:-2,:].reshape(-1,1)-addtrainPredict.reshape(-1,1), trainPredict[timesteps:-2,:].reshape(-1,1)], axis=1)
result_te = np.concatenate([addtestPredict.reshape(-1,1), teY[timesteps:-2,:].reshape(-1,1), teY[timesteps:-2,:].reshape(-1,1)-addtestPredict.reshape(-1,1), testPredict[timesteps:-2,:].reshape(-1,1)], axis=1)

result_tr_df = pd.DataFrame(result_tr, columns=['Predicted Value','Real Value','Forecasting Error', 'Initial Predicted Value'])
result_te_df = pd.DataFrame(result_te, columns=['Predicted Value','Real Value','Forecasting Error', 'Initial Predicted Value'])

In [54]:
error_tr_FFEL = trvaY[timesteps:-2,:].reshape(-1,1)-addtrainPredict.reshape(-1,1)
error_tr_basic = trvaY[timesteps:-2,:].reshape(-1,1)-trainPredict[timesteps:-2,:].reshape(-1,1)
error_te_FFEL = teY[timesteps:-2,:].reshape(-1,1)-addtestPredict.reshape(-1,1)
error_te_basic = teY[timesteps:-2,:].reshape(-1,1)-testPredict[timesteps:-2,:].reshape(-1,1)

In [55]:
np.mean(error_tr_FFEL), np.var(error_tr_FFEL), np.mean(error_tr_basic), np.var(error_tr_basic), np.mean(error_te_FFEL),np.var(error_te_FFEL), np.mean(error_te_basic), np.var(error_te_basic)

(0.012406698850194922,
 0.0035503282092661033,
 0.011721027340419807,
 0.011909348497146503,
 0.010097205411530795,
 0.0042666178094028775,
 0.012302805598353127,
 0.012685604994840286)

In [56]:
result_tr_df.to_csv('training_ramp_result_df_fh12.csv')
result_te_df.to_csv('test_ramp_result_df_fh12.csv')